In [ ]:
import seaborn as sns
import pandas as pd
# import numpy as np
import scipy
import networkx
# import matplotlib
%pylab inline

from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import cross_val_score

# Question 01: Propensity Score matching

### 01 naive analysis
Let's start by loading in our data

In [ ]:
df_lalonde = pd.read_csv(r'lalonde.csv')
df_lalonde.head()

Let's have a quick look at our data with describe(). We can already note a few aspects of interest
* We're looking mostly at young people, an average of 27 years with 1st and 3rd quartiles at 20 and 32 respectively
* About 60% of the subjects don't have a degree
* About 40% of the subjects are black and 10% hispanic
* If we look at the mean revenue in 1974 and 75 we can see a big fall, which isn't too surprising as this was right during the oil crisis and the [73-75 recession](https://en.wikipedia.org/wiki/1973%E2%80%9375_recession). By 1976 this recession was over and GDP reached it's pre-1973 level, explaining the much higher salaries in 78

In [ ]:
df_lalonde.describe()

Let's have a look at how many of the subjects actually had a job in '78 (ie. income greater than 0).

It looks the vast majority were employed:

In [ ]:
# Adding column with employment status and looking at unemployment
df_lalonde['em78'] = df_lalonde['re78']
df_lalonde.em78 = df_lalonde.em78.apply(lambda x: x if x == 0 else 1)
df_lalonde.em78.plot(kind='hist', bins=2)

In [ ]:
df_lalonde.head()

We'll make two `DataFrames`, one with the treated subjects and the other with non-treated subjects. Then let's `describe()` the revenues in '78 of these two groups.

It looks like most metrics are very close, even the standard deviation is similar. It's hard to say from these analysis of the two groups if the treatment had any effect

In [ ]:
df_treat = df_lalonde[df_lalonde['treat']==1]
df_notreat = df_lalonde[df_lalonde['treat']==0]

df_new = pd.DataFrame(index=df_treat.describe().index, columns=['treat re78','notreat re78'])
# df_new = df_treat.re78.describe()
df_new.loc[:, 'treat re78'] = df_treat.describe()['re78'].values
df_new.loc[:, 'notreat re78'] = df_notreat.describe()['re78'].values
df_new

Maybe a histogram or cumulative histogram will give us a better insight into these two groups.

Unfortunately it doesn't look there is much of a difference between the groups here either. There are a few outliers in the treated group with very high salaries, but that's it. It also appears that the revenu follows a power law.

In [ ]:
fig, axes = plt.subplots(1, 2)
df_treat.re78.plot(kind='hist', ax=axes[0], title='Histogram', label='treated', alpha=0.5, figsize=(16,8))
df_notreat.re78.plot(kind='hist', ax=axes[0], label='untreated', alpha=0.5)
axes[0].legend()

df_treat.re78.plot(kind='hist', cumulative=True, ax=axes[1], title='Cumulative histogram', label='treated', alpha=0.5)
df_notreat.re78.plot(kind='hist', cumulative=True, ax=axes[1], label='untreated', alpha=0.5)

In [ ]:
fig, axes = plt.subplots(1, 2)
df_treat[df_treat.re78 != 0].re78.plot(kind='hist', ax=axes[0], title='Histogram', label='treated', alpha=0.5, figsize=(16,8))
df_notreat[df_notreat.re78 != 0].re78.plot(kind='hist', ax=axes[0], label='untreated', alpha=0.5)
axes[0].legend()

df_treat[df_treat.re78 != 0].re78.plot(kind='hist', cumulative=True, ax=axes[1], title='Cumulative histogram', label='treated', alpha=0.5)
df_notreat[df_notreat.re78 != 0].re78.plot(kind='hist', cumulative=True, ax=axes[1], label='untreated', alpha=0.5)

Just looking at the histograms, means and medians are not very helpful to draw a clear cut answer of whether the treatment has an effect or not. We have a lot more samples for the non-treated population too, which makes it harder to compare simple histograms

Given the histograms, the revenue is clearly not normally distributed. Furthermore, the samples are of different sizes, so we would have to use the Mann-Whitney U test to try and estimate if the treated and untreated populations have a different revenue. As we can see from the returned p value close to 1, we cannot discard the null hypothesis that these two samples are drawn from the same population. If we follow the naive reasoning we may then conclude that these populations are in fact the same and that the treatment had no effect

In [ ]:
stat, pval = scipy.stats.mannwhitneyu(df_treat.re78.values, df_treat.re78.values, alternative='two-sided')
print(stat,'\n',pval)

### 02 A closer look at the data
We're now interested in distributions of variables within our two sets. For a quick look over our set let's plot a heatmap of correlation between all our variables. What we're really interested in is whether a subject is treated or not is correlated to any other of the subject's features. This is represented by only one row/column from our heatmap, but additional correlations may appear that are of interest.

In [ ]:
sns.heatmap(df_lalonde.corr().values, center=0, 
            xticklabels=df_lalonde.columns[1:], yticklabels=df_lalonde.columns[1:])

There's a very large correlation between treatment and whether a subject is black. The other two important correlations are marital status and revenue in '74. Let's plot these three attributes and see what they're like between our two groups.

We can see some interesting trends if we start correlating the different characteristics within the groups.

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(16, 6))
# todo: choose better color scheme

sns.heatmap(df_treat.corr(method='spearman').values, ax=axes[0], center=0, 
            xticklabels=df_treat.columns[1:], yticklabels=df_lalonde.columns[1:])
sns.heatmap(df_notreat.corr(method='spearman').values, ax=axes[1], center=0, 
            xticklabels=df_notreat.columns[1:], yticklabels=df_lalonde.columns[1:])

In [ ]:
fig, axes = plt.subplots(1, 3)
df_treat.black.plot(kind='hist', bins=2, ax=axes[0], alpha=0.5, 
                    title='black', label='treated', figsize=(16, 8))
df_notreat.black.plot(kind='hist', bins=2, ax=axes[0], alpha=0.5, label='non-treated')
axes[0].legend()
df_treat.married.plot(kind='hist', bins=2, ax=axes[1], alpha=0.5,
                      title='married', label='treated')
df_notreat.married.plot(kind='hist', bins=2, ax=axes[1], alpha=0.5, label='non-treated')

df_treat.re74.plot(kind='hist', bins=2, ax=axes[2], alpha=0.5,
                     title='re74', label='treated')
df_notreat.re74.plot(kind='hist', bins=2, ax=axes[2], alpha=0.5, label='non-treated')

Let's have a quick check that we're not being misled by our correlation coefficient by having a look at an attribute which doesn't seem to be correlated with treatment, for example hispanic.
Looks good, there appears to be about the same distribution of hispanics and non-hispanics in our two groups

In [ ]:
df_treat.hispan.plot(kind='hist', bins=2, alpha=0.5,
                     title='hispan', label='treated')
df_notreat.hispan.plot(kind='hist', bins=2, alpha=0.5, label='non-treated').legend()

### 03 A propensity score model
To account for any variations in feature between treated and control sets, we'd now like to estimate the probability of treatment of a subject given their features.

In [ ]:
df_lalonde.columns
features = ['age', 'educ', 'black', 'hispan', 'married', 'nodegree']
x = pd.get_dummies(df_lalonde[features])
y = df_lalonde['re78']

logistic = LogisticRegression()
# logistic.fit(x,y)